In [1]:
import pandas as pd
import numpy as np
import pprint

In [2]:
# base variables
csv_files = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS"]

BASE_DIR = "~/Research/wheelchair/data/raw/Max"
OUTPUT_DIR = "~/Research/wheelchair/data/processed/"

# variables for glove types
materials = ["HYB", "PLA"]

### Helper functions and config

In [3]:
def mean_positive(x):
    positives = x[x > 0]
    if len(x) == 0:
        return np.nan
    return positives.mean()

def mean_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.mean()

def peak_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.min()



# Building the parameter dictionary
def build_parameter_dictionary():
    xy_force_params = {
        "tangential_force":"N" , 
        "radial_force": "N", 
        "axle_force": "N"
    }
    z_params = {
        "moment_z_R": "Nm", 
        "power_z": "W"
    }
    
    sides = ["L", "R", "avg"]

    xy_force_params = {
        "tangential_force":"N" , 
        "radial_force": "N", 
        "axle_force": "N"
    }
    z_params = {
        "moment_z_R": "Nm", 
        "power_z": "W"
    }
    
    sides = ["L", "R", "avg"]
    
    # build up a dictionary containing the parameters we want to calculate
    agg_dict = {}
    for col, unit in xy_force_params.items():
        for side in sides:
            col_name = f"{col}_{side}" 
            og_column = f"{col_name}[{unit}]"
            agg_dict[f"{col_name}_pos[{unit}]"] = (og_column, mean_positive)
            agg_dict[f"{col_name}_neg[{unit}]"] = (og_column, mean_negative)
            agg_dict[f"{col_name}_pos_peak[{unit}]"] = (og_column, "max")
            agg_dict[f"{col_name}_neg_peak[{unit}]"] = (og_column, peak_negative)
    
    for col, unit in z_params.items():
        og_column = f"{col}[{unit}]"
        agg_dict[f"{col}_pos[{unit}]"] = (og_column, mean_positive)
        agg_dict[f"{col}_neg[{unit}]"] = (og_column, mean_negative)
        agg_dict[f"{col}_pos_peak[{unit}]"] = (og_column, "max")
        agg_dict[f"{col}_neg_peak[{unit}]"] = (og_column, peak_negative)
    return agg_dict

### Clean the data
Also add variables

In [12]:
# load and clean the data
def load_data(path):
    raw_df = pd.read_csv(path)

    # only use named columns and when the hand cycle is touching
    df = raw_df.drop(columns=[c for c in raw_df.columns if "Unnamed" in c])
    df = df[df['theta_cop_R[deg]'].notna()]

    return df

def compute_force_columns(df):
    # add the power[W] calculation for future calculations down the line
    df['power_z[W]'] = df['gyro_z_R[rad/s]']*df['moment_z_R[Nm]']
    
    # add the power[W] calculation for future calculations down the line
    df['total_force_R[N]'] = np.sqrt(
        df["tangential_force_R[N]"]**2 +
        df["radial_force_R[N]"]**2 +
        df["axle_force_R[N]"]**2
    )
    
    df['total_force_L[N]'] =  np.sqrt(
        df["tangential_force_L[N]"]**2 +
        df["radial_force_L[N]"]**2 +
        df["axle_force_L[N]"]**2
    )
    
    # Set missing values to 0
    # i.e. if they lift their hand up earlier, the force is not recorded, and is set to 0
    df['total_force_R[N]'] = df['total_force_R[N]'].fillna(0)
    df['total_force_L[N]'] = df['total_force_L[N]'].fillna(0)
    df['total_force_average[N]'] = (df['total_force_R[N]'] + df['total_force_L[N]'])/2.0

    return df


def compute_avg_forces(df):
    # Add the average values across the R and L sides too
    xy_params = ["tangential_force", "radial_force", "axle_force"]

    for param in xy_params:
        avg_param_name = f"{param}_avg[N]"
        l_param_name =f"{param}_L[N]"
        r_param_name=f"{param}_R[N]"
        df[avg_param_name]=(df[l_param_name] + df[r_param_name]) / 2.0
    return df

def aggregate_per_cycle(df):
    agg_dict = build_parameter_dictionary()
    return df.groupby("cycle[count]").agg(**agg_dict)

In [13]:
def run_all(material, initials):
    input_file = f"{BASE_DIR}/{material}/{initials}25{material}.csv"
    output_file = f"{OUTPUT_DIR}/{material}/{initials}25{material}_per_cycle.csv"

    print(f"Processing file for {initials} with gloves {material}")
    df = load_data(input_file)
    df = compute_force_columns(df)
    df = compute_avg_forces(df)
    agg_df = aggregate_per_cycle(df)

    agg_df.to_csv(output_file)

    print(f"Saved → {output_file}")
    
    return agg_df

In [14]:
for material in materials:
    for initials in csv_files:
        run_all(material, initials)

Processing file for ALP with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/ALP25HYB_per_cycle.csv
Processing file for BS with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/BS25HYB_per_cycle.csv
Processing file for DR with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/DR25HYB_per_cycle.csv
Processing file for EC with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/EC25HYB_per_cycle.csv
Processing file for HD with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/HD25HYB_per_cycle.csv
Processing file for JF with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/JF25HYB_per_cycle.csv
Processing file for JR with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/JR25HYB_per_cycle.csv
Processing file for SS with gloves HYB
Saved → ~/Research/wheelchair/data/processed//HYB/SS25HYB_per_cycle.csv
Processing file for ALP with gloves PLA
Saved → ~/Research/wheelchair/data/processed//PLA/ALP25PLA_per_cycle.c